<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/model_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

**PREPARING DATASET**

Model Architecture

In [2]:
base_model = keras.applications.ResNet50(include_top=False, weights=None, 
                                            input_shape=(224, 224, 3), pooling='avg')
#base_model.trainable = False
x = base_model.output
prediction = keras.layers.Dense(1, activation="linear")(x)
model = keras.Model(inputs = base_model.input, outputs = prediction)

Model Config

In [ ]:
opt = keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mse", optimizer=opt,
              metrics=['mse', 'mape', tfa.metrics.RSquare(name="R2", y_shape=(1,))])

Train Model

In [ ]:
print("[INFO] training model...")
model.fit(x=trainImagesX, y=trainY, 
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

Model Summary

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model.png", show_shapes=False)